In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
# !pip install surt
# !pip install validators

In [3]:
# Two handy libraries for checking if a URL is valid or not
# import surt
import validators

## SEC-Registered

In [43]:
df_sec = pd.read_csv('data/sec/IA_FIRM_SEC_Feed_02_15_2023.csv')

/tmp/ipykernel_161773/3704932919.py:1: DtypeWarning: Columns (49,217,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sec = pd.read_csv('data/sec/IA_FIRM_SEC_Feed_02_15_2023.csv')


In [44]:
sorted(df_sec.columns.to_list())

['Filing.@Dt',
 'Filing.@FormVrsn',
 'FormInfo.Part1A.Item1.@Q1F5',
 'FormInfo.Part1A.Item1.@Q1I',
 'FormInfo.Part1A.Item1.@Q1M',
 'FormInfo.Part1A.Item1.@Q1N',
 'FormInfo.Part1A.Item1.@Q1O',
 'FormInfo.Part1A.Item1.@Q1ODesc',
 'FormInfo.Part1A.Item1.@Q1P',
 'FormInfo.Part1A.Item1.WebAddrs',
 'FormInfo.Part1A.Item1.WebAddrs.WebAddr',
 'FormInfo.Part1A.Item10A.@Q10A',
 'FormInfo.Part1A.Item11.@Q11',
 'FormInfo.Part1A.Item11A.@Q11A1',
 'FormInfo.Part1A.Item11A.@Q11A2',
 'FormInfo.Part1A.Item11B.@Q11B1',
 'FormInfo.Part1A.Item11B.@Q11B2',
 'FormInfo.Part1A.Item11C.@Q11C1',
 'FormInfo.Part1A.Item11C.@Q11C2',
 'FormInfo.Part1A.Item11C.@Q11C3',
 'FormInfo.Part1A.Item11C.@Q11C4',
 'FormInfo.Part1A.Item11C.@Q11C5',
 'FormInfo.Part1A.Item11D.@Q11D1',
 'FormInfo.Part1A.Item11D.@Q11D2',
 'FormInfo.Part1A.Item11D.@Q11D3',
 'FormInfo.Part1A.Item11D.@Q11D4',
 'FormInfo.Part1A.Item11D.@Q11D5',
 'FormInfo.Part1A.Item11E.@Q11E1',
 'FormInfo.Part1A.Item11E.@Q11E2',
 'FormInfo.Part1A.Item11E.@Q11E3',
 'F

Number of Advisers -- everything

In [7]:
df_sec.shape[0]

21075

In [12]:
df_sec['large_or_mid_sized'] = (df_sec['FormInfo.Part1A.Item2A.@Q2A1'] == 'Y') | (df_sec['FormInfo.Part1A.Item2A.@Q2A2'] == 'Y')

In [13]:
df_sec['robo'] = df_sec['FormInfo.Part1A.Item2A.@Q2A11'] == 'Y'

Number of Advisers -- just the "regular" firms

In [15]:
df_sec_reg = df_sec[df_sec['large_or_mid_sized']].copy()
df_sec_reg.shape[0]

13794

Number of Robo Advisers

In [16]:
df_sec_robo = df_sec[df_sec['robo']].copy()
df_sec_robo.shape[0]

269

## State-Registered Advisers 

In [41]:
df_state = pd.read_csv('data/sec/IA_FIRM_STATE_Feed_02_15_2023.csv')

/tmp/ipykernel_161773/3518042785.py:1: DtypeWarning: Columns (35,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,270,277,299,306,309,310,313) have mixed types. Specify dtype option on import or set low_memory=False.
  df_state = pd.read_csv('data/sec/IA_FIRM_STATE_Feed_02_15_2023.csv')


In [42]:
sorted(df_state.columns.to_list())

['ERA.Rgltrs',
 'ERA.Rgltrs.Rgltr',
 'ERA.Rgltrs.Rgltr.@Cd',
 'ERA.Rgltrs.Rgltr.@Dt',
 'ERA.Rgltrs.Rgltr.@St',
 'Filing.@Dt',
 'Filing.@FormVrsn',
 'FormInfo.Part1A.Item1.@Q1F5',
 'FormInfo.Part1A.Item1.@Q1I',
 'FormInfo.Part1A.Item1.@Q1M',
 'FormInfo.Part1A.Item1.@Q1N',
 'FormInfo.Part1A.Item1.@Q1O',
 'FormInfo.Part1A.Item1.@Q1ODESC',
 'FormInfo.Part1A.Item1.@Q1P',
 'FormInfo.Part1A.Item1.WebAddrs',
 'FormInfo.Part1A.Item1.WebAddrs.WebAddr',
 'FormInfo.Part1A.Item10A.@Q10A',
 'FormInfo.Part1A.Item11.@Q11',
 'FormInfo.Part1A.Item11A.@Q11A1',
 'FormInfo.Part1A.Item11A.@Q11A2',
 'FormInfo.Part1A.Item11B.@Q11B1',
 'FormInfo.Part1A.Item11B.@Q11B2',
 'FormInfo.Part1A.Item11C.@Q11C1',
 'FormInfo.Part1A.Item11C.@Q11C2',
 'FormInfo.Part1A.Item11C.@Q11C3',
 'FormInfo.Part1A.Item11C.@Q11C4',
 'FormInfo.Part1A.Item11C.@Q11C5',
 'FormInfo.Part1A.Item11D.@Q11D1',
 'FormInfo.Part1A.Item11D.@Q11D2',
 'FormInfo.Part1A.Item11D.@Q11D3',
 'FormInfo.Part1A.Item11D.@Q11D4',
 'FormInfo.Part1A.Item11D.@Q11D5

Some Advisers are "Exempt Reporting Advisers", they are typically VCs 
https://learn.angellist.com/articles/exempt-reporting-advisor

In [23]:
#df_state_exempt = df_state.dropna(subset=[
#    "ERA.Rgltrs", "ERA.Rgltrs.Rgltr", "ERA.Rgltrs.Rgltr.@Cd", "ERA.Rgltrs.Rgltr.@Dt", "ERA.Rgltrs.Rgltr.@St",
#    ], how="all")

In [26]:
from functools import reduce
import operator as op

In [39]:
exemption_cols = ["ERA.Rgltrs", "ERA.Rgltrs.Rgltr", "ERA.Rgltrs.Rgltr.@Cd", "ERA.Rgltrs.Rgltr.@Dt", "ERA.Rgltrs.Rgltr.@St"]
state_exempt = reduce(op.or_, [df_state[col].notnull() for col in exemption_cols])
df_state["is_exempt"] = state_exempt

In [30]:
df_state_exempt = df_state[state_exempt].copy()
df_state_reg = df_state[~state_exempt].copy()

In [31]:
df_state_exempt_old = df_state[(df_state['ERA.Rgltrs'].notnull() |
                            df_state['ERA.Rgltrs.Rgltr'].notnull() |
                            df_state['ERA.Rgltrs.Rgltr.@Cd'].notnull() |
                            df_state['ERA.Rgltrs.Rgltr.@Dt'].notnull() |
                            df_state['ERA.Rgltrs.Rgltr.@St'].notnull() )].copy()

In [32]:
df_state_exempt.shape, df_state_exempt_old.shape

((3870, 314), (3870, 314))

In [33]:
df_state_reg_old =df_state[(df_state['ERA.Rgltrs'].isnull() &
                        df_state['ERA.Rgltrs.Rgltr'].isnull() &
                        df_state['ERA.Rgltrs.Rgltr.@Cd'].isnull() &
                        df_state['ERA.Rgltrs.Rgltr.@Dt'].isnull() &
                        df_state['ERA.Rgltrs.Rgltr.@St'].isnull() )].copy()

In [34]:
df_state_reg.shape, df_state_reg_old.shape

((17274, 314), (17274, 314))

In [35]:
df_state_reg.shape[0] + df_state_exempt.shape[0] == df_state.shape[0]

True

# Get All Web Addresses

In [ ]:
df_state

In [36]:
def get_all_urls(df, valid_only=True):
    df_urls = df.loc[df['FormInfo.Part1A.Item1.WebAddrs.WebAddr'].notnull(),
                    ['Info.@FirmCrdNb',
                     'FormInfo.Part1A.Item1.WebAddrs.WebAddr']
                    ]
    df_urls.rename(columns={'FormInfo.Part1A.Item1.WebAddrs.WebAddr': 'url'}, inplace=True)
    df_urls['url'] = df_urls['url'].apply(lambda x: eval(x.lower()) if ('[' in x and ']' in x) else x.lower())
    df_urls = df_urls.explode('url').reset_index(drop=True)
    df_urls['url_valid'] = df_urls['url'].apply(validators.url)
    if valid_only:
        df_urls = df_urls.loc[df_urls['url_valid'] == True,:]
    return df_urls

In [37]:
df_sec_reg_urls = get_all_urls(df_sec_reg, valid_only=False)

In [38]:
df_sec_reg_urls

,Info.@FirmCrdNb,url,url_valid
0,282351,https://facebook.com/thefausergroup,True
1,282351,http://fausergroup.com/,True
2,145800,https://www.linkedin.com/company/freestate-adv...,True
3,145800,http://www.freestateadvisors.com,True
4,285650,http://www.morseasset.com,True
...,...,...,...
48620,319062,http://www.azimuthcap.com,True
48621,319062,https://www.linkedin.com/in/azimuth-capital-ma...,True
48622,319062,https://focusfinancialpartners.com,True
48623,319062,https://www.linkedin.com/company/2064318/,True


---

In [109]:
firm_nb = set(df_sec['Info.@FirmCrdNb']) & set(df_state['Info.@FirmCrdNb'])

In [111]:
len(firm_nb)

1759

In [115]:
df_state[df_state["Info.@FirmCrdNb"].isin(firm_nb)].iloc[:10].sort_values("Info.@FirmCrdNb")

,Info.@FirmCrdNb,Info.@BusNm,Info.@LegalNm,Info.@UmbrRgstn,MainAddr.@Strt1,MainAddr.@Strt2,MainAddr.@City,MainAddr.@State,MainAddr.@Cntry,MainAddr.@PostlCd,...,FormInfo.Part1B.ItemJ.@Q1BJ2BPfs,FormInfo.Part1B.ItemJ.@Q1BJ2BNone,FormInfo.Part1A.Item9E.@Q9E,ERA.Rgltrs.Rgltr,FormInfo.Part1B.ItemH.@Q1B2HNScrtsNvsmtAm,FormInfo.Part1A.Item1.@Q1ODESC,FormInfo.Part1A.Item5D.@Q5D1MOth,FormInfo.Part1A.Item5H.@Q5HMT500,FormInfo.Part1A.Item5C.@Q5C1Oth,FormInfo.Part1A.Item5D.@Q5D2MOth
112,159488,AEB CAPITAL,"AEB CAPITAL, LLC",NaN,1701 PENNSYLVANIA AVE NW STE 200,NaN,WASHINGTON,DC,United States,20006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,161098,OKAPI VENTURE CAPITAL,"OKAPI VENTURE CAPITAL, LLC",NaN,1600 NEWPORT CENTER DRIVE,SUITE 1600,NEWPORT BEACH,CA,United States,92660,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,171863,MOSAIK PARTNERS,"MOSAIK PARTNERS, LP",NaN,555 CALIFORNIA STREET,24TH FLOOR,SAN FRANCISCO,CA,United States,94104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,172513,OWL VENTURES,"OWL VENTURES, LLC",NaN,855 EL CAMINO REAL STE 13A-354,NaN,PALO ALTO,CA,United States,94301,...,NaN,NaN,NaN,NaN,NaN,$1 billion - $10 billion,NaN,NaN,NaN,NaN
160,206518,"WALSER WEALTH MANAGEMENT COMPANY, A LIMITED LI...","WALSER WEALTH MANAGEMENT COMPANY, A LIMITED LI...",N,777 S. HARBOUR ISLAND BLVD,STE 440,TAMPA,FL,United States,33602,...,NaN,NaN,NaN,NaN,38174054.0,NaN,NaN,NaN,NaN,NaN
107,226615,"MEGHALAYA PARTNERS, L.P.","MEGHALAYA PARTNERS, L.P.",NaN,500 WEST PUTNAM AVENUE,SUITE 400,GREENWICH,CT,United States,06830,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,292360,LONGEVITY FUNDS 2 LLC,LONGEVITY FUNDS 2 LLC,NaN,555 BRYANT STREET,#517,PALO ALTO,CA,United States,94301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,298061,ZZG CAPITAL LLC,ZZG CAPITAL LLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,298342,IDEO COLAB VENTURES LLC,IDEO COLAB VENTURES LLC,NaN,501 THE EMBARCADERO,PIER 28 ANNEX,SAN FRANCISCO,CA,United States,94105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,299671,"DEFINE VENTURES MANAGEMENT, LLC","DEFINE VENTURES MANAGEMENT, LLC",NaN,97 ARGUELLO BLVD,NaN,SAN FRANCISCO,CA,United States,94129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
df_sec[df_sec["Info.@FirmCrdNb"].isin(firm_nb)].iloc[:10].sort_values("Info.@FirmCrdNb")

,Info.@SECRgnCD,Info.@FirmCrdNb,Info.@SECNb,Info.@BusNm,Info.@LegalNm,Info.@UmbrRgstn,MainAddr.@Strt1,MainAddr.@Strt2,MainAddr.@City,MainAddr.@State,...,FormInfo.Part1A.Item5D.@Q5D2E,FormInfo.Part1A.Item5D.@Q5D2F,FormInfo.Part1A.Item5D.@Q5D2G,FormInfo.Part1A.Item5D.@Q5D2H,FormInfo.Part1A.Item5D.@Q5D2I,FormInfo.Part1A.Item5D.@Q5D2J,FormInfo.Part1A.Item5D.@Q5D2K,FormInfo.Part1A.Item5D.@Q5D2L,FormInfo.Part1A.Item5D.@Q5D2M,FormInfo.Part1A.Item5C.@Q5C1Oth
102,PLRO,159488,802-78855,AEB CAPITAL,"AEB CAPITAL, LLC",NaN,1701 PENNSYLVANIA AVE NW STE 200,NaN,WASHINGTON,DC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,LARO,161098,802-74499,OKAPI VENTURE CAPITAL,"OKAPI VENTURE CAPITAL, LLC",NaN,1600 NEWPORT CENTER DRIVE,SUITE 1600,NEWPORT BEACH,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SFRO,171863,802-123465,MOSAIK PARTNERS,"MOSAIK PARTNERS, LP",NaN,555 CALIFORNIA STREET,24TH FLOOR,SAN FRANCISCO,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,SFRO,172513,802-80272,OWL VENTURES,"OWL VENTURES, LLC",NaN,855 EL CAMINO REAL STE 13A-354,NaN,PALO ALTO,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,MIRO,206518,801-127246,"WALSER WEALTH MANAGEMENT COMPANY, A LIMITED LI...","WALSER WEALTH MANAGEMENT COMPANY, A LIMITED LI...",N,777 S. HARBOUR ISLAND BLVD,STE 440,TAMPA,FL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,BRO,226615,802-115159,"MEGHALAYA PARTNERS, L.P.","MEGHALAYA PARTNERS, L.P.",NaN,500 WEST PUTNAM AVENUE,SUITE 400,GREENWICH,CT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,SFRO,292360,802-112647,LONGEVITY FUNDS 2 LLC,LONGEVITY FUNDS 2 LLC,NaN,555 BRYANT STREET,#517,PALO ALTO,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,NYRO,298061,802-124823,ZZG CAPITAL LLC,ZZG CAPITAL LLC,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,SFRO,298342,802-126171,IDEO COLAB VENTURES LLC,IDEO COLAB VENTURES LLC,NaN,501 THE EMBARCADERO,PIER 28 ANNEX,SAN FRANCISCO,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,SFRO,299671,802-114577,"DEFINE VENTURES MANAGEMENT, LLC","DEFINE VENTURES MANAGEMENT, LLC",NaN,97 ARGUELLO BLVD,NaN,SAN FRANCISCO,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
df_sec["Info.@SECNb"].isnull().sum()

2

In [106]:
df_state['Info.@FirmCrdNb'].nunique()

21144

In [107]:
df_state.shape

(21144, 314)

In [ ]:
df_sec

In [53]:
n_unique_crd_nb = len(set(df_sec['Info.@FirmCrdNb']) | set(df_state['Info.@FirmCrdNb']))
print("Unique Firms:", n_unique_crd_nb)

Unique Firms: 40460


In [103]:
df_sec['Info.@FirmCrdNb'].value_counts().max()

1

In [55]:
import json

In [65]:
validators.url("https://asdf.com")

True

In [66]:
bool(_)

True

In [74]:
all_web_addrs = list(df_sec['FormInfo.Part1A.Item1.WebAddrs.WebAddr'].dropna()) + list(df_state['FormInfo.Part1A.Item1.WebAddrs.WebAddr'].dropna())

In [78]:
# data engineering podcast

In [79]:
all_urls = []
for web_addr in all_web_addrs:
    web_addr = web_addr.lower()
    if web_addr.startswith("["):
        web_addr = web_addr.replace("'", '"')
        web_addr = web_addr.removeprefix("[").removesuffix("]")
        candidates = [url.removeprefix('"').removesuffix('"') for url in web_addr.split(", ")]
    else:
        candidates = [web_addr]
    all_urls.extend(candidates)

In [86]:
from urllib.parse import urlparse
from collections import defaultdict

In [97]:
domains = defaultdict(int)
for url in all_urls:
    if validators.url(url):
        domain = urlparse(url).netloc
        if domain.startswith("www."):
            domain = domain.removeprefix("www.")
        elif domain.startswith("m."):
            domain = domain.removeprefix("m.")
        elif domain.startswith("mobile."):
            domain = domain.removeprefix("mobile.")
        domains[domain] += 1

In [ ]:
40460

In [100]:
pd.Series(domains).sort_values(ascending=False)[:50] / 40460

linkedin.com                     0.497331
facebook.com                     0.229486
twitter.com                      0.204993
instagram.com                    0.066807
youtube.com                      0.054251
medium.com                       0.006574
vimeo.com                        0.005413
us.rbcwealthmanagement.com       0.005091
tiktok.com                       0.003930
angel.co                         0.002991
soundcloud.com                   0.002694
pinterest.com                    0.002595
focusfinancialpartners.com       0.002175
rcm.rockco.com                   0.001977
advisor.hilltopsecurities.com    0.001928
yelp.com                         0.001829
crunchbase.com                   0.001705
feeonlynetwork.com               0.001656
investdavenport.com              0.001582
plus.google.com                  0.001483
open.spotify.com                 0.001434
reddit.com                       0.001409
napfa.org                        0.001236
thefinancialhq.com               0

In [101]:
pd.Series(domains).sort_values(ascending=False)[:50]

linkedin.com                     20122
facebook.com                      9285
twitter.com                       8294
instagram.com                     2703
youtube.com                       2195
medium.com                         266
vimeo.com                          219
us.rbcwealthmanagement.com         206
tiktok.com                         159
angel.co                           121
soundcloud.com                     109
pinterest.com                      105
focusfinancialpartners.com          88
rcm.rockco.com                      80
advisor.hilltopsecurities.com       78
yelp.com                            74
crunchbase.com                      69
feeonlynetwork.com                  67
investdavenport.com                 64
plus.google.com                     60
open.spotify.com                    58
reddit.com                          57
napfa.org                           50
thefinancialhq.com                  48
prudential.com                      46
linkedin                 

In [89]:
urlparse(url).netloc

'facebook.com'

In [80]:
bad_urls = [url for url in all_urls if not validators.url(url)]

In [81]:
len(bad_urls)

494

In [83]:
bad_urls[:50]

['https://www.linkedin.com/showcase/ardian-women"s-club/',
 'http://www.contourventures.com;   https://www.linkedin.com/company/contour-venture-partners/',
 'https://medium.com@primaryvc',
 'http://www.linkedin.com - dif capital partners',
 'https://linkedin.com  acg wealth management',
 'https://twitter.com @acgrva',
 "http://www.guidestone.org  (organization web site that contains links to all guidestone affiliates' websites)",
 'www.instagram.com/redewealth',
 'https://www.youtube.com/channel/uccu1ztphtapsgg -mpsxrng',
 'https://www.twitter.com/the pinnaclefg',
 'https://line.me/r/ti/p/%40gpt0697p \\xa0messaging app: need scan qr code',
 'https://www.facebook.com/flagship private wealth/',
 'http://www.facebook.com/the planning center/',
 'https://www.facebook.com /burnsestateplanning estateplanning',
 'https://www.acerra-financial.com./',
 'https://www.ma-financial',
 'https://www.mi4na1kteam',
 'https://www.facebook.com/american-financial-tax-strategies-inc- 530263377039835/',
 'h

In [68]:
len(all_web_addrs)

21389

In [84]:
len(all_urls)

94393

In [75]:
all_web_addrs[:10]

["['HTTPS://FACEBOOK.COM/THEFAUSERGROUP', 'HTTP://FAUSERGROUP.COM/']",
 "['https://www.linkedin.com/company/freestate-advisors-llc', 'HTTP://WWW.FREESTATEADVISORS.COM']",
 'HTTP://WWW.MORSEASSET.COM',
 "['HTTPS://WWW.FACEBOOK.COM/ALBIONFINANCIALGROUP/', 'HTTPS://TWITTER.COM/ALBIONFINGRP', 'HTTP://WWW.ALBIONFINANCIAL.COM', 'HTTPS://WWW.LINKEDIN.COM/COMPANY/ALBION-FINANCIAL-GROUP', 'https://www.instagram.com/womenofalbion', 'http://linkedin.com/showcase/women-of-albion']",
 'HTTP://CRAWFORDFP.COM',
 "['HTTPS://WWW.ALTUSCAPITALPARTNERS.COM', 'HTTPS://WWW.LINKEDIN.COM/COMPANY/ALTUS-CAPITAL-PARTNERS/']",
 "['HTTPS://WWW.VALUESFIRST.COM', 'HTTPS://WWW.VALUESFIRSTADVISORS.COM', 'HTTPS://WWW.LINKEDIN.COM/COMPANY/VALUES-FIRST-ADVISORS']",
 "['https://twitter.com/mosaikpartners', 'https://www.linkedin.com/company/mosaik-partners/', 'HTTP://WWW.MOSAIKPARTNERS.COM', 'https://www.facebook.com/MosaikPartners/']",
 "['HTTP://WWW.NATREALESTATEDEVELOPMENT.COM/', 'HTTP://WWW.NATADVISORS.COM', 'HTTPS://W

In [ ]:
def extract_url(web_addr):
    web_addr = web_addr.lower()
    if web_addr.startswith("["):
        candidates = json.loads(web_addr.replace("'", '"'))
    else:
        candidates = [web_addr]
    

In [61]:
json.loads(df_sec['FormInfo.Part1A.Item1.WebAddrs.WebAddr'].str.replace("'", '"'))

['HTTPS://FACEBOOK.COM/THEFAUSERGROUP', 'HTTP://FAUSERGROUP.COM/']

In [58]:
json.loads(df_sec['FormInfo.Part1A.Item1.WebAddrs.WebAddr'].iloc[1])

JSONDecodeError: Expecting value: line 1 column 2 (char 1)

0        146099
1        289221
2         23940
3        155220
4        171863
          ...  
21139    324692
21140    324710
21141    324715
21142    324698
21143    324726
Name: Info.@FirmCrdNb, Length: 21144, dtype: int64

In [47]:
set(df_sec.columns) - set(df_state.columns)

{'FormInfo.Part1A.Item1.@Q1ODesc',
 'FormInfo.Part1A.Item2A.@Q2A1',
 'FormInfo.Part1A.Item2A.@Q2A10',
 'FormInfo.Part1A.Item2A.@Q2A11',
 'FormInfo.Part1A.Item2A.@Q2A12',
 'FormInfo.Part1A.Item2A.@Q2A13',
 'FormInfo.Part1A.Item2A.@Q2A2',
 'FormInfo.Part1A.Item2A.@Q2A4',
 'FormInfo.Part1A.Item2A.@Q2A5',
 'FormInfo.Part1A.Item2A.@Q2A6',
 'FormInfo.Part1A.Item2A.@Q2A7',
 'FormInfo.Part1A.Item2A.@Q2A8',
 'FormInfo.Part1A.Item2A.@Q2A9',
 'FormInfo.Part1A.Item2B.@Q2B1',
 'FormInfo.Part1A.Item2B.@Q2B2',
 'FormInfo.Part1A.Item2B.@Q2B3',
 'Info.@SECRgnCD',
 'NoticeFiled.States.@Dt',
 'NoticeFiled.States.@RgltrCd',
 'NoticeFiled.States.@St',
 'Rgstn.@Dt',
 'Rgstn.@FirmType',
 'Rgstn.@St'}

In [48]:
set(df_state.columns) - set(df_sec.columns)

{'ERA.Rgltrs',
 'ERA.Rgltrs.Rgltr',
 'ERA.Rgltrs.Rgltr.@Cd',
 'ERA.Rgltrs.Rgltr.@Dt',
 'ERA.Rgltrs.Rgltr.@St',
 'FormInfo.Part1A.Item1.@Q1ODESC',
 'FormInfo.Part1A.Item5D.@Q5D1MOth',
 'FormInfo.Part1A.Item5D.@Q5D2MOth',
 'FormInfo.Part1B.DsclrQstns.@Q1B2C',
 'FormInfo.Part1B.DsclrQstns.@Q1B2D',
 'FormInfo.Part1B.DsclrQstns.@Q1B2E1',
 'FormInfo.Part1B.DsclrQstns.@Q1B2E2',
 'FormInfo.Part1B.DsclrQstns.@Q1B2E3',
 'FormInfo.Part1B.DsclrQstns.@Q1B2E4',
 'FormInfo.Part1B.DsclrQstns.@Q1B2E5',
 'FormInfo.Part1B.DsclrQstns.@Q1B2F1',
 'FormInfo.Part1B.DsclrQstns.@Q1B2F2',
 'FormInfo.Part1B.DsclrQstns.@Q1B2F3',
 'FormInfo.Part1B.DsclrQstns.@Q1B2F4',
 'FormInfo.Part1B.DsclrQstns.@Q1B2F5',
 'FormInfo.Part1B.Item2.@Q1B2B1',
 'FormInfo.Part1B.Item2.@Q1B2B2',
 'FormInfo.Part1B.Item2.@Q1B2B4',
 'FormInfo.Part1B.ItemG.@Q1B2G2',
 'FormInfo.Part1B.ItemG.@Q1BG1IssueScrts',
 'FormInfo.Part1B.ItemG.@Q1BG1PoolInvmt',
 'FormInfo.Part1B.ItemG.@Q1BG1ReAdvsr',
 'FormInfo.Part1B.ItemG.@Q1BG1TaxPrprr',
 'FormInfo.P